Use this cell for all your imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly
from scipy.sparse import csr_matrix, coo_matrix, diags
from scipy.sparse.linalg import norm

You will be working with the file data.txt. Use this cell to load its content into the appropriate data structure.

In [ ]:
filename = "data.txt"
graph = {}

def read_file():
    # Open the file and read its content
    with open(filename, 'r') as f:
        for line in f:
            # Parse the line and extract the nodes
            nodes = line.strip().split()
            if len(nodes) != 2:
                continue
            from_node = int(nodes[0])
            to_node = int(nodes[1])

            # Add the nodes to the graph dictionary
            if from_node not in graph:
                graph[from_node] = []
            if to_node not in graph:
                graph[to_node] = []
            graph[from_node].append(to_node)
    return graph



In [ ]:
res = read_file()

In [ ]:
res.items()

dict_items([(1001, [9304045, 9308122, 9309097, 9311042, 9401139, 9404151, 9407087, 9408099, 9501030, 9503124, 9504090, 9504145, 9505025, 9505054, 9505105, 9505162, 9506048, 9506112, 9506144, 9507050, 9507158, 9508094, 9508155, 9510142, 9510225, 9510234, 9511030, 9511171, 9601108, 9602022, 9602114, 9603003, 9603150, 9603161, 9603167, 9605184, 9605222, 9606017, 9606040, 9607163, 9607207, 9608086, 9609070, 9609071, 9609239, 9611137, 9612108, 9701162, 9702094, 9702155, 9702198, 9703082, 9703166, 9704097, 9705030, 9705044, 9705104, 9705220, 9706005, 9707014, 9707042, 9707049, 9710230, 9711036, 9711104, 9712028, 9712042, 9802194, 9805056, 9805206, 9806094, 9810188, 9811217, 9905036, 9907041, 9908007, 9908144, 9909108, 9909120, 9909229, 9910238, 9910248, 9910268]), (9304045, [9204040]), (9308122, [9203084, 9204035, 9205041, 9207049, 9207111, 9301042, 9301043]), (9309097, [9203084, 9205041, 9301042, 9301043]), (9311042, [9301042, 9301043]), (9401139, [9201015, 9201040, 9201047, 9202046, 920205

Use this cell to normalize the matrix as you see fit.

In [ ]:
def to_coo_matrix(data_dict):
    # Create lists to store the row, column, and data values for the COO matrix
    row = []
    col = []
    data = []

    # Loop over the dictionary and populate the COO matrix
    row_index = 0
    for key, values in data_dict.items():
        for value in values:
            row.append(row_index)
            col.append(value)
            data.append(1)
        row_index += 1
    # Create the COO matrix
    matrix_coo = coo_matrix((data, (row, col)))#, shape=(len(set(row)), len(set(col))))
    row_sums = np.array(matrix_coo.sum(axis=1)).flatten()

    # create a diagonal matrix of row sums
    D = diags(row_sums)
    # compute D^-1 * matrix_coo
    matrix_coo = D.power(-1)*matrix_coo

    return matrix_coo

In [ ]:
normalized_matrix = to_coo_matrix(res)

/shared-libs/python3.9/py/lib/python3.9/site-packages/scipy/sparse/_data.py:117: RuntimeWarning: divide by zero encountered in reciprocal
  return self._with_data(data ** n)


In [ ]:
print(normalized_matrix)

  (0, 9910268)	0.012048192771084338
  (0, 9910248)	0.012048192771084338
  (0, 9910238)	0.012048192771084338
  (0, 9909229)	0.012048192771084338
  (0, 9909120)	0.012048192771084338
  (0, 9909108)	0.012048192771084338
  (0, 9908144)	0.012048192771084338
  (0, 9908007)	0.012048192771084338
  (0, 9907041)	0.012048192771084338
  (0, 9905036)	0.012048192771084338
  (0, 9811217)	0.012048192771084338
  (0, 9810188)	0.012048192771084338
  (0, 9806094)	0.012048192771084338
  (0, 9805206)	0.012048192771084338
  (0, 9805056)	0.012048192771084338
  (0, 9802194)	0.012048192771084338
  (0, 9712042)	0.012048192771084338
  (0, 9712028)	0.012048192771084338
  (0, 9711104)	0.012048192771084338
  (0, 9711036)	0.012048192771084338
  (0, 9710230)	0.012048192771084338
  (0, 9707049)	0.012048192771084338
  (0, 9707042)	0.012048192771084338
  (0, 9707014)	0.012048192771084338
  (0, 9706005)	0.012048192771084338
  :	:
  (27764, 9606105)	0.058823529411764705
  (27764, 9508161)	0.058823529411764705
  (27764, 9502

Apply the PageRank algorithm to the matrix you created.

In [ ]:
#  https://www.youtube.com/watch?v=urKLHNhUEQ0

def pageRankAlgorithm(P, tol):
    n = P.shape[1]
    max_iter = 100
    r = np.ones(n) / n
    for i in range(max_iter):
        r_new = P.multiply(r)
        if norm(r_new - r) < tol:
            return r
        r = r_new
        return r

In [ ]:
pageRankResults = pageRankAlgorithm(normalized_matrix, tol=0.001)

Output a list of the 100 most important papers along with their importance

In [ ]:
def sort_coo(m):
    tuples = zip(m.row, m.col, m.data)
    results =  sorted(tuples, key=lambda x: (x[2]), reverse=True)[:100]
    data = np.array([res[2] for res in results ])
    row = np.array([res[0] for res in results ])
    col = np.array([res[1] for res in results ])
    n, m = np.max(row), np.max(col)
    return coo_matrix((data, (row, col)))#, shape=(n, m))#.toarray()
    return data, rows, cols

papers_and_importance = pageRankResults

In [ ]:
papers_and_importance_resulsts= sort_coo(papers_and_importance)
papers_and_importance_resulsts

<2123x9912243 sparse matrix of type '<class 'numpy.float64'>'
	with 100 stored elements in COOrdinate format>

Visualize your results

In [ ]:
papers_and_importance_resulsts.data

array([1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e-07,
       1.0088482e-07, 1.0088482e-07, 1.0088482e-07, 1.0088482e

In [ ]:
import plotly.express as px
fig = px.scatter(color=papers_and_importance_resulsts.data, x=papers_and_importance_resulsts.row, y=papers_and_importance_resulsts.col)
fig.show()

In [ ]:
#please see plot.png